# Transformacion de la tabla Customer  y ecritura en la capa silver


In [0]:
df_customers=spark.table("gizmobox.bronze.py_customers")

In [0]:
display(df_customers.limit(10))

REMOVE RECORDS WITH NULL customer_id

In [0]:
# Vamos a eliminar los id que tienen nulos
display(df_customers.where(df_customers.customer_id.isNotNull()).limit(10))

In [0]:
df_customers=df_customers.where(df_customers.customer_id.isNotNull())

In [0]:
# Vamos a eliminar los id que tienen nulos
display(df_customers.filter('customer_id IS NOT NULL').limit(10))

REMOVE EXACT DUPLICATE RECORDS

In [0]:
df_customers=df_customers.distinct()


Utilizando la funcion DROPDUPLICATE

In [0]:
df_customers=df_customers.dropDuplicates()

REMOVE DUPLICATE RECORDS BASED ON CREATED_TIMESTAMP

In [0]:
display(df_customers.orderBy(df_customers.customer_id)
)

In [0]:
from pyspark.sql.functions import max
df_customers_1=df_customers.groupBy(df_customers.customer_id)\
                .agg(max("created_timestamp").alias("mas_created_timestamp"))

In [0]:
display(df_customers_1.orderBy("customer_id"))

In [0]:
display(df_customer_filter.limit(3))

In [0]:
    df_join=(
        df_customers_1.join(df_customers, (df_customers_1.customer_id==df_customers.customer_id) & (df_customers_1.mas_created_timestamp==df_customers.created_timestamp), 'inner')
     
        .select(df_customers["*"])

  
    )


In [0]:
display(df_join)

COMO CAMBIAR EL TIPO DE DATOS


In [0]:

df_customer_casted=(
          df_join.select(df_join.created_timestamp.cast("timestamp"),
                         df_join.customer_id.cast("integer"),
                         df_join.customer_name,
                         df_join.date_of_birth.cast("date"),
                         df_join.email,
                         df_join.member_since.cast("date"),
                         df_join.telephone )
  
  
  )

In [0]:
display(df_customer_casted.limit(10))

In [0]:
df_customer_casted.writeTo("gizmobox.silver.py_customers").createOrReplace()